In [1]:
import cv2
import numpy as np

# Function to add Gaussian noise to an image
def add_gaussian_noise(image, mean=0, sigma=30):
    # Generate Gaussian noise with specified mean and sigma
    row, col = image.shape
    gauss = np.random.normal(mean, sigma, (row, col))
    # Add the noise to the image
    noisy_image = image + gauss
    # Clip values to ensure they are within 0-255 range
    noisy_image = np.clip(noisy_image, 0, 255)
    # Convert back to uint8
    noisy_image = noisy_image.astype(np.uint8)
    return noisy_image

# Function to implement Otsu's algorithm for thresholding
def otsu_threshold(image):
    # Calculate histogram
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])
    hist_norm = hist.ravel() / hist.sum()
    best_threshold = -1
    best_variance = 0
    
    # Iterate over all possible threshold values
    for threshold in range(256):
        # Calculate background and foreground probabilities and means
        w0 = np.sum(hist_norm[:threshold])/np.sum(hist_norm[:256])
        if w0 == 0:
            continue
        mean0 = np.sum(np.arange(threshold) * hist_norm[:threshold]) / np.sum(hist_norm[:threshold])
        
        w1 = np.sum(hist_norm[threshold:])/np.sum(hist_norm[:256])
        if w1 == 0:
            continue
        mean1 = np.sum(np.arange(threshold, 256) * hist_norm[threshold:]) / np.sum(hist_norm[threshold:])
         
        # Calculate within class variance
        within_class_variance = w0 * w1 *(mean0-mean1)**2
        
        # Update best threshold if variance is improved
        if within_class_variance > best_variance:
            best_threshold = threshold
            best_variance = within_class_variance
    
    # Apply thresholding using the best threshold value
    _, otsu_image = cv2.threshold(image, best_threshold, 255, cv2.THRESH_BINARY)
    return otsu_image

# Define image dimensions
height = 300
width = 300

# Create a blank white image
image = np.ones((height, width), dtype=np.uint8) * 255

# Draw a circle
cv2.circle(image, (height//3, width//2), 50, (0, 0, 0), -1)  # Circle with center (height//3, width//2) and radius 50

# Draw a triangle
pts = np.array([[2*height//3, width//3], [2*height//3 + 60, width//3 + 100], [2*height//3 - 60, width//3 + 100]], np.int32)
cv2.fillPoly(image, [pts], (155,155,155))  # Triangle with vertices defined in pts list

# Add Gaussian noise to the image
noisy_image = add_gaussian_noise(image)

# Implement Otsu's algorithm for thresholding
otsu_image = otsu_threshold(noisy_image)

# Display the original image, noisy image, and Otsu thresholded image
cv2.imshow('Original Image', image)
cv2.imshow('Noisy Image', noisy_image)
cv2.imshow('Otsu Thresholded Image', otsu_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
